## Classifying Handwritten Digits

In [ ]:
import matplotlib.pyplot as plt
import sklearn.datasets
import sklearn.model_selection
import sklearn.metrics

We use the toy digit dataset provided by scikit-learn.  

(You may also find it fun later to try your hand at the MNIST dataset, one of the classic initial problems for budding machine-learning practicioners.)

In [ ]:
d = sklearn.datasets.load_digits()

In [ ]:
print(d.DESCR)

In [ ]:
x = d.data
y = d.target

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
x[0]

In [ ]:
y[0]

The samples consist of 64 features, one for each pixel value of an 8x8 image array.  We can reshape the sample into an 8x8 array in order to visualize it.

In [ ]:
sample = x[4].reshape(8,8)
plt.imshow(sample, cmap='binary')

In [ ]:
for i in range(100):
    plt.subplot(10,10,i+1)
    sample = x[i].reshape(8,8)
    plt.imshow(sample, cmap='binary')

## Random Forest

In [ ]:
import sklearn.ensemble
rf_classifier = sklearn.ensemble.RandomForestClassifier(max_depth=15)

In [ ]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(
        x, y, test_size=0.2, random_state=42)

One catch to watch out for in splitting up your data into training and test sets:  stratification.

Let's say you have a dataset that has 90% cat images and 10% dog images.  If you split your data and end up with 99% cats in your training data and 1% dogs, you'll be training your model on an unrepresentative sample.  (Sampling issues like this can be much more consequential and damaging than distinguishing cats from dogs!)

In [ ]:
plt.hist(y, width=0.5)

In [ ]:
plt.hist(y_train, width=0.5)

Here the difference in percentages is noticeable but not too significant by eye.  Nevertheless, we can stratify our split properly by including the "stratify" parameter and assigning it our target variable.

In [ ]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(
        x, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
plt.hist(y, width=0.5)

In [ ]:
plt.hist(y_train, width=0.5)

In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np

In [ ]:
depth_range = range(1, 20)
depth_scores = []
for d in depth_range:
    rf_classifier = sklearn.ensemble.RandomForestClassifier(max_depth=d)
    acc_score = cross_val_score(rf_classifier,
                           x_train,
                           y_train, 
                           cv=5, 
                           scoring='accuracy')
    depth_scores.append(acc_score.mean())
plt.scatter(depth_range, depth_scores)
plt.xlabel('Value of Max Depth for RF Classifier')
plt.ylabel('Cross-Validated Accuracy')

In [ ]:
rf_classifier = sklearn.ensemble.RandomForestClassifier(max_depth=16)

In [ ]:
rf_classifier.fit(x_train, y_train)

In [ ]:
rf_classifier.predict(x_train[[7]])

In [ ]:
y_train[7]

In [ ]:
y_pred = rf_classifier.predict(x_test)

In [ ]:
cm = sklearn.metrics.confusion_matrix(y_test, y_pred)
cm

In [ ]:
plt.imshow(cm,cmap='binary')
plt.xlabel('predicted')
plt.ylabel('actual')

In contrast with binary classification, calculating precision and recall (and etc) for multi-class classification problems can be computed in slightly different ways depending on how one does averaging. 

A macro-average will compute the metric independently for each class and then take the average (hence treating all classes equally), whereas a micro-average will aggregate the contributions of all classes to compute the average metric. 

In a multi-class classification setup, micro-average is preferable if you suspect there might be class imbalance (i.e you may have many more examples of one class than of other classes).

In [ ]:
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, y_pred):.2%}")
print(f"Precision: {sklearn.metrics.precision_score(y_test, y_pred, average='micro'):.2%}")
print(f"Recall: {sklearn.metrics.recall_score(y_test, y_pred, average='micro'):.2%}")

In [ ]:
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, y_pred):.2%}")
print(f"Precision: {sklearn.metrics.precision_score(y_test, y_pred, average='macro'):.2%}")
print(f"Recall: {sklearn.metrics.recall_score(y_test, y_pred, average='macro'):.2%}")

## Logistic Regression

In [ ]:
import sklearn.linear_model
lr_classifier = sklearn.linear_model.LogisticRegression()

In [ ]:
lr_classifier.fit(x_train, y_train)

It will not be uncommon for you to run into scenarios in which you encounter errors when trying to train models.

In such cases, they can be fruitful opportunities to consult the documentation and learn more about various training options.

Here, the error message gives us clues about potentially insightful documentation.

To fast-forward, it will be useful here for Logistic Regression if we rescale our sample data from being integer values over [0:16] to being continuous values scaled to have a normal distribution of values -> the sklearn StandardScaler will rescale the features to have 0 mean and unit variance.

In [ ]:
import sklearn.preprocessing

In [ ]:
scaler = sklearn.preprocessing.StandardScaler()

In [ ]:
x_scaled = scaler.fit_transform(x_train)

In [ ]:
x_scaled[0]

Here's the difference in image between original and rescaled.

In [ ]:
sample = x_train[7].reshape(8,8)
plt.imshow(sample,cmap='binary')

In [ ]:
sample = x_scaled[7].reshape(8,8)
plt.imshow(sample,cmap='binary')

In [ ]:
for i in range(100):
    plt.subplot(10,10,i+1)
    sample = x_scaled[i].reshape(8,8)
    plt.imshow(sample, cmap='binary')

In [ ]:
lr_classifier.fit(x_scaled, y_train)

In [ ]:
lr_classifier.predict(x_scaled[[7]])

In [ ]:
y_train[7]

Our classifier was trained on scaled data, so we must scale any new data similarly (though we only need to do the transform now, not the fit.)

In [ ]:
x_test_scaled = scaler.transform(x_test)

In [ ]:
y_pred = lr_classifier.predict(x_test_scaled)

In [ ]:
cm = sklearn.metrics.confusion_matrix(y_test, y_pred)
cm

In [ ]:
plt.imshow(cm,cmap='binary')
plt.xlabel('predicted')
plt.ylabel('actual')

In [ ]:
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, y_pred):.2%}")
print(f"Precision: {sklearn.metrics.precision_score(y_test, y_pred, average='micro'):.2%}")
print(f"Recall: {sklearn.metrics.recall_score(y_test, y_pred, average='micro'):.2%}")

In [ ]:
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, y_pred):.2%}")
print(f"Precision: {sklearn.metrics.precision_score(y_test, y_pred, average='macro'):.2%}")
print(f"Recall: {sklearn.metrics.recall_score(y_test, y_pred, average='macro'):.2%}")

## Naive Bayes

In [ ]:
import sklearn.naive_bayes
nb_classifier = sklearn.naive_bayes.MultinomialNB()

In [ ]:
nb_classifier.fit(x_train, y_train)

In [ ]:
y_pred = nb_classifier.predict(x_test)

In [ ]:
cm = sklearn.metrics.confusion_matrix(y_test, y_pred)
cm

In [ ]:
plt.imshow(cm,cmap='binary')
plt.xlabel('predicted')
plt.ylabel('actual')

In [ ]:
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, y_pred):.2%}")
print(f"Precision: {sklearn.metrics.precision_score(y_test, y_pred, average='micro'):.2%}")
print(f"Recall: {sklearn.metrics.recall_score(y_test, y_pred, average='micro'):.2%}")

In [ ]:
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, y_pred):.2%}")
print(f"Precision: {sklearn.metrics.precision_score(y_test, y_pred, average='macro'):.2%}")
print(f"Recall: {sklearn.metrics.recall_score(y_test, y_pred, average='macro'):.2%}")

We could try doing the scaling here too to see if that improves the result.

In [ ]:
scaler = sklearn.preprocessing.StandardScaler()

In [ ]:
x_scaled = sklearn.preprocessing.StandardScaler().fit_transform(x_train)

In [ ]:
nb_classifier.fit(x_scaled, y_train)

For MultinomialNB, we actually can't use negative values in the training data.  A multinomial distribution is appropriate for discrete positive values (and actually indicates that the original scale is likely best).

If we use MinMaxScaler, though, we'll get a similar standardization of the scaling, but now on the range of [0,1].

In [ ]:
scaler = sklearn.preprocessing.MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [ ]:
x_train_scaled[0]

In [ ]:
nb_classifier.fit(x_train_scaled, y_train)

In [ ]:
y_pred = nb_classifier.predict(x_test_scaled)

In [ ]:
cm = sklearn.metrics.confusion_matrix(y_test, y_pred)
cm

In [ ]:
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, y_pred):.2%}")
print(f"Precision: {sklearn.metrics.precision_score(y_test, y_pred, average='micro'):.2%}")
print(f"Recall: {sklearn.metrics.recall_score(y_test, y_pred, average='micro'):.2%}")

In [ ]:
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, y_pred):.2%}")
print(f"Precision: {sklearn.metrics.precision_score(y_test, y_pred, average='macro'):.2%}")
print(f"Recall: {sklearn.metrics.recall_score(y_test, y_pred, average='macro'):.2%}")

## Extra section: Binary Classification and the Precision/Recall Trade-Off

In classification there is a trade-off between optimizing precision and optimizing recall.

The following will allow you to quantify and visualize that trade-off, as well as make plots of the ROC curves (Receiver Operating Characteristic).

ROC is useful for binary classification, when you have a strict population of false-negatives, true-positives, and etc.  Precision and recall are also more easily conceptualized in binary classification, when there is no ambiguity about handling multi-class classification.  Therefore, the below does classification for the classes Fives and Not-Fives.  (You can easily check for other numbers too by changing the 5 to, say, 9).

In [ ]:
y_five = (y == 5)

In [ ]:
y_five

In [ ]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(
        x, y_five, test_size=0.2, random_state=42, stratify=y_five)

In [ ]:
import numpy as np

In [ ]:
np.sum(y_five) / y_five.shape

In [ ]:
np.sum(y_test) / y_test.shape

In [ ]:
np.sum(y_train) / y_train.shape

In [ ]:
# Random Forest
rf_classifier = sklearn.ensemble.RandomForestClassifier()
rf_classifier.fit(x_train, y_train)
rf_y_pred = rf_classifier.predict(x_test)
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, rf_y_pred):.2%}")
print(f"RF Precision: {sklearn.metrics.precision_score(y_test, rf_y_pred):.2%}")
print(f"RF Recall: {sklearn.metrics.recall_score(y_test, rf_y_pred):.2%}")
cm = sklearn.metrics.confusion_matrix(y_test, rf_y_pred)
print(cm)

# Logistic Regression
lr_classifier = sklearn.linear_model.LogisticRegression()
x_train_scaled = sklearn.preprocessing.StandardScaler().fit_transform(x_train)
x_test_scaled = sklearn.preprocessing.StandardScaler().fit_transform(x_test)
lr_classifier.fit(x_train_scaled, y_train)
lr_y_pred = lr_classifier.predict(x_test_scaled)
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, lr_y_pred):.2%}")
print(f"LR Precision: {sklearn.metrics.precision_score(y_test, lr_y_pred):.2%}")
print(f"LR Recall: {sklearn.metrics.recall_score(y_test, lr_y_pred):.2%}")
cm = sklearn.metrics.confusion_matrix(y_test, lr_y_pred)
print(cm)

# Multinomial Naive Bayes
nb_classifier = sklearn.naive_bayes.MultinomialNB()
nb_classifier.fit(x_train, y_train)
nb_y_pred = nb_classifier.predict(x_test)
print(f"Accuracy: {sklearn.metrics.accuracy_score(y_test, nb_y_pred):.2%}")
print(f"NB Precision: {sklearn.metrics.precision_score(y_test, nb_y_pred):.2%}")
print(f"NB Recall: {sklearn.metrics.recall_score(y_test, nb_y_pred):.2%}")
cm = sklearn.metrics.confusion_matrix(y_test, nb_y_pred)
print(cm)

In [ ]:
# rf_y_pred_train = rf_classifier.predict_proba(x_train)
# lr_y_pred_train = lr_classifier.predict_proba(x_train_scaled)
# nb_y_pred_train = nb_classifier.predict_proba(x_train)

# rf_falsePositiveRate, rf_truePositiveRate, rf_thresholds = sklearn.metrics.roc_curve(y_train, rf_y_pred_train[:,1])
# lr_falsePositiveRate, lr_truePositiveRate, lr_thresholds = sklearn.metrics.roc_curve(y_train, lr_y_pred_train[:,1])
# nb_falsePositiveRate, nb_truePositiveRate, nb_thresholds = sklearn.metrics.roc_curve(y_train, nb_y_pred_train[:,1])

In [ ]:
rf_y_scores = sklearn.model_selection.cross_val_predict(rf_classifier, x_train, y_train, cv=3,
                             method="predict_proba")
lr_y_scores = sklearn.model_selection.cross_val_predict(lr_classifier, x_train_scaled, y_train, cv=3,
                             method="predict_proba")
nb_y_scores = sklearn.model_selection.cross_val_predict(nb_classifier, x_train, y_train, cv=3,
                             method="predict_proba")

rf_falsePositiveRate, rf_truePositiveRate, rf_thresholds = sklearn.metrics.roc_curve(y_train, rf_y_scores[:,1])
lr_falsePositiveRate, lr_truePositiveRate, lr_thresholds = sklearn.metrics.roc_curve(y_train, lr_y_scores[:,1])
nb_falsePositiveRate, nb_truePositiveRate, nb_thresholds = sklearn.metrics.roc_curve(y_train, nb_y_scores[:,1])

In [ ]:
plt.figure(figsize=(6, 5))  # extra code – not needed, just formatting
plt.plot(rf_falsePositiveRate, rf_truePositiveRate, linewidth=2, color='black', label="RF ROC curve")
plt.plot(lr_falsePositiveRate, lr_truePositiveRate, linewidth=2, color='blue', label="LR ROC curve")
plt.plot(nb_falsePositiveRate, nb_truePositiveRate, linewidth=2, color='green', label="NB ROC curve")
plt.plot([0, 1], [0, 1], 'k:', label="Random classifier's ROC curve")
plt.legend()
plt.show()

In [ ]:
# rf_y_pred_train = rf_classifier.predict_proba(x_train)
# rf_precisions, rf_recalls, rf_thresholds = sklearn.metrics.precision_recall_curve(y_train, rf_y_pred_train[:,1])

# lr_y_pred_train = lr_classifier.predict_proba(x_train_scaled)
# lr_precisions, lr_recalls, lr_thresholds = sklearn.metrics.precision_recall_curve(y_train, lr_y_pred_train[:,1])

# nb_y_pred_train = nb_classifier.predict_proba(x_train)
# nb_precisions, nb_recalls, nb_thresholds = sklearn.metrics.precision_recall_curve(y_train, nb_y_pred_train[:,1])

In [ ]:
rf_precisions, rf_recalls, rf_thresholds = sklearn.metrics.precision_recall_curve(y_train, rf_y_scores[:,1])

lr_precisions, lr_recalls, lr_thresholds = sklearn.metrics.precision_recall_curve(y_train, lr_y_scores[:,1])

nb_precisions, nb_recalls, nb_thresholds = sklearn.metrics.precision_recall_curve(y_train, nb_y_scores[:,1])

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(rf_thresholds, rf_precisions[:-1], "b--", label="RF Precision", linewidth=2)
plt.plot(rf_thresholds, rf_recalls[:-1], "g-", label="RF Recall", linewidth=2)
plt.axis([0, 1, 0, 1.1])
plt.xlabel("Threshold")
plt.legend()

plt.figure(figsize=(8, 4))
plt.plot(lr_thresholds, lr_precisions[:-1], "b--", label="LR Precision", linewidth=2)
plt.plot(lr_thresholds, lr_recalls[:-1], "g-", label="LR Recall", linewidth=2)
plt.axis([0, 1, 0, 1.1])
plt.xlabel("Threshold")
plt.legend()

plt.figure(figsize=(8, 4))
plt.plot(nb_thresholds, nb_precisions[:-1], "b--", label="NB Precision", linewidth=2)
plt.plot(nb_thresholds, nb_recalls[:-1], "g-", label="NB Recall", linewidth=2)
plt.axis([0, 1, 0, 1.1])
plt.xlabel("Threshold")
plt.legend()

plt.show()